In [ ]:
!pip install opencv-python

In [7]:
import cv2
from PIL import Image
import numpy as np

In [4]:
video = cv2.VideoCapture(0)
while video.isOpened():
    success,frame = video.read()
    if success:
        cv2.imshow('camera',frame)
        key = cv2.waitKey(1)&0xff
        if key == 27: # ESC키를 누르면 중지
            break
video.release()
cv2.destroyAllWindows()

In [8]:
classes = []
with open('coco.names.txt','r') as f:
    classes = [ line.strip() for line in f.readlines() ]    
colors = np.random.uniform(0,255,size=(len(classes),3) )

model = cv2.dnn.readNet('yolov3.weights','yolov3.cfg')   
layer_names = model.getLayerNames()
out_layers = [ layer_names[i-1] for i in model.getUnconnectedOutLayers()]

In [10]:
def process_video():
    video = cv2.VideoCapture(0)
    while video.isOpened():
        success,img = video.read()
        if success:
            cv2.imshow('camera',img)
            key = cv2.waitKey(1)&0xff
            if key == 27: # ESC키를 누르면 중지
                break
            height,width,channels =  img.shape
            blob =  cv2.dnn.blobFromImage(img,1.0/256,(448,448),(0,0,0),swapRB=True,crop=False)
            model.setInput(blob)
            outputs = model.forward(out_layers)

            class_ids,confidences,boxes = [],[],[]
            # outputs  모델에 대한 검출한 객체의 정보를 담고 있음
            for output_ in outputs:
              for vec85 in output_:  # vec85 객체의 정보를 담고 있는 벡터
                scores = vec85[5:]  # 최고 부류 확률이 0.5를 넘는 바운딩 박스  클래스의 확률 점수
                class_id = np.argmax(scores)
                confidence=scores[class_id] # 선택된 클래스에 해당하는 신뢰도 값을 가져옮
                if confidence>0.5: # 신뢰도가 50% 이상인 경우만 취함
                    centerx,centery=int(vec85[0]*width),int(vec85[1]*height)  # 중심좌표를 이미지 크기에 맞게 변환
                    w,h=int(vec85[2]*width),int(vec85[3]*height) # 너비와 높이를 조정
                    x,y=int(centerx-w/2),int(centery-h/2) # 좌상단 좌표를 계산
                    boxes.append([x,y,w,h]) # 감지된 객체의 좌표와 크기를 boxes 리스트에 저장
                    confidences.append(float(confidence))
                    class_ids.append(class_id)
            # Non-Maximum Supperssino(NMS)을 사용
            # 겹치는 박스를 제거, 겹치는 박스중에 가장 신뢰도가 높은 박스들의 좌표만 남겨둔다       
            indexes=cv2.dnn.NMSBoxes(boxes,confidences,0.5,0.4) 

            for i in range(len(boxes)):
              if i in indexes:
                x,y,w,h=boxes[i]
                text=str(classes[class_ids[i]])+'%.3f'%confidences[i]
                cv2.rectangle(img,(x,y),(x+w,y+h),colors[class_ids[i]],2)
                cv2.putText(img,text,(x,y+30),cv2.FONT_HERSHEY_PLAIN,2,colors[class_ids[i]],2)
            if 0 in class_ids:  # 사람을 검출( 'person')
                print("person detect!!!")                
        
    video.release()
    cv2.destroyAllWindows()

In [11]:
process_video()

TypeError: argument of type 'numpy.int64' is not iterable